## 导入库函数

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

## 数据处理

In [29]:
# 读取所有工作表
def read_all_sheet(excel_file='bio_train_trans.xlsx',target_rows=9,writefile='step_9_trans.xlsx'):
    all_sheets = pd.read_excel(excel_file, sheet_name=None)
    # 创建一个字典来存储处理后的数据框
    processed_sheets = {}
    # 处理每个工作表
    for sheet_name, df in all_sheets.items():
        # 如果行数超过9行，只保留前9行
        if len(df) > target_rows:
            df = df.iloc[:target_rows]
        
        # 如果行数少于9行，用前一行数据填充
        while len(df) < target_rows:
            # 获取最后一行数据
            last_row = df.iloc[-1:]
            # 将这行数据添加到数据框末尾
            df = pd.concat([df, last_row], ignore_index=True)
        
        processed_sheets[sheet_name] = df

    # 将处理后的数据保存到新的Excel文件
    with pd.ExcelWriter(writefile) as writer:
        for sheet_name, df in processed_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print("所有工作表已处理完成！每个工作表现在都是9行。")
    
read_all_sheet('bio_train_trans.xlsx',9,'step_9_trans.xlsx')

所有工作表已处理完成！每个工作表现在都是9行。


In [22]:
def process_sheet(df):
    """处理单个sheet，添加下一时间步的酸钠值作为新特征"""
    # 创建下一时间步的酸钠列
    df['next_酸钠'] = df['酸钠'].shift(-1)
    df['next_残糖'] = df['残糖g/dl'].shift(-1)
    return df

def process_sheets(excel_path):
    
    # 读取所有sheet
    excel_file = pd.ExcelFile(excel_path)
    sheet_names = excel_file.sheet_names
    
    # 创建新的Excel文件
    output_path = 'processed_' + os.path.basename(excel_path)
    
    # 处理每个sheet
    with pd.ExcelWriter(output_path) as writer:
        for sheet_name in sheet_names:
            # 读取当前sheet
            df = pd.read_excel(excel_path, sheet_name=sheet_name)
            
            # 处理数据
            processed_df = process_sheet(df)
            
            # 保存到新的Excel文件
            processed_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"处理完成！结果已保存到: {output_path}")
    

process_sheets('step_9_trans.xlsx')

处理完成！结果已保存到: processed_step_9_trans.xlsx


In [9]:
# 合并所有的sheet
# 读取Excel文件中的所有工作表
excel_file = 'processed_bio_train_trans.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)
# 初始化一个空的DataFrame来存储合并后的数据
merged_df = pd.DataFrame()

# 遍历所有工作表并合并
for sheet_name, df in all_sheets.items():
    # 假设发酵周期列名为'发酵周期'或'Time'，请根据实际情况调整
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.concat([merged_df, df], ignore_index=True)

# 保存合并后的数据到新的Excel文件
output_file = 'merged_step_9_trans.xlsx'
merged_df.to_excel(output_file, index=False)


In [13]:
merged_df.head(10) 

,发酵周期/h,酶活,酸钠,残糖g/dl,菌浓ml/50ml,菌浓g/50ml,PH值,罐压,风量L/h,转速r/min,溶氧,温度,碱重kg,重量KG,next_酸钠,next_残糖
0,0.00,11.10000,0.65,34.00,2.0,0.0143,5.22,0.071,1500.0,550.0,99.1,38.0,13.41,33.2,2.36,32.00
1,4.00,42.20000,2.36,32.00,8.0,0.0206,5.21,0.069,1500.0,550.0,91.7,38.0,13.26,31.2,5.42,28.50
2,8.00,120.00000,5.42,28.50,15.0,0.0270,5.18,0.070,1500.0,550.0,54.5,38.0,12.82,31.1,9.31,23.75
3,12.00,225.00000,9.31,23.75,17.0,0.0249,5.18,0.070,1500.0,550.0,35.9,38.0,12.15,31.7,13.29,19.20
4,16.00,314.42432,13.29,19.20,18.0,0.0241,5.18,0.070,1500.0,550.0,27.5,38.0,11.49,31.9,17.56,14.90
5,20.00,327.75680,17.56,14.90,20.0,0.0247,5.18,0.070,1500.0,550.0,20.8,38.0,10.75,32.4,22.11,12.00
6,24.00,337.80000,22.11,12.00,22.0,0.0340,5.18,0.069,1500.0,550.0,17.0,38.0,10.03,32.9,25.40,6.80
7,28.00,516.60000,25.40,6.80,22.0,0.0350,5.17,0.069,1500.0,550.0,14.6,38.0,9.34,33.2,29.26,1.82
8,32.55,677.70000,29.26,1.82,33.0,0.0468,5.18,0.071,1500.0,550.0,84.5,37.9,8.55,33.7,NaN,NaN
9,0.00,41.10000,1.82,34.50,10.0,0.0112,5.30,0.086,1500.0,550.0,77.7,38.0,8.51,31.7,4.71,31.75


### 缺失值填补

In [ ]:
# 填补缺失值

# # 随机森林填补缺失值
# def fill_missing_with_rf(df):
#     # 对每个包含缺失值的列进行填充
#     for column in df.columns[df.isnull().any()]:
#         # 分离包含缺失值和不包含缺失值的行
#         known = df[df[column].notnull()]
#         unknown = df[df[column].isnull()]
        
#         if len(known) == 0 or len(unknown) == 0:
#             continue
            
#         # 准备特征（使用其他列作为特征）
#         features = [x for x in df.columns if x != column]
        
#         # 去除特征中包含缺失值的行
#         known_features = known[features].fillna(known[features].mean())
        
#         # 训练随机森林模型
#         rf = RandomForestRegressor(n_estimators=100, random_state=42)
#         rf.fit(known_features, known[column])
        
#         # 预测缺失值
#         unknown_features = unknown[features].fillna(known[features].mean())
#         predictions = rf.predict(unknown_features)
        
#         # 填充缺失值
#         df.loc[df[column].isnull(), column] = predictions
    
#     return df

# # 应用随机森林填充缺失值
# merged_df = fill_missing_with_rf(merged_df)

# # 保存处理后的数据
# output_file = 'filled_merged_bio_train.xlsx'
# merged_df.to_excel(output_file, index=False)

In [14]:
#使用0进行填补
imp_0 = SimpleImputer(missing_values=np.nan, strategy="constant",fill_value=0)
filled_0_merged = imp_0.fit_transform(merged_df)
# 转换回 DataFrame
filled_0_merged = pd.DataFrame(filled_0_merged, columns=merged_df.columns, index=merged_df.index)

In [15]:
filled_0_merged.head(10)

,发酵周期/h,酶活,酸钠,残糖g/dl,菌浓ml/50ml,菌浓g/50ml,PH值,罐压,风量L/h,转速r/min,溶氧,温度,碱重kg,重量KG,next_酸钠,next_残糖
0,0.00,11.10000,0.65,34.00,2.0,0.0143,5.22,0.071,1500.0,550.0,99.1,38.0,13.41,33.2,2.36,32.00
1,4.00,42.20000,2.36,32.00,8.0,0.0206,5.21,0.069,1500.0,550.0,91.7,38.0,13.26,31.2,5.42,28.50
2,8.00,120.00000,5.42,28.50,15.0,0.0270,5.18,0.070,1500.0,550.0,54.5,38.0,12.82,31.1,9.31,23.75
3,12.00,225.00000,9.31,23.75,17.0,0.0249,5.18,0.070,1500.0,550.0,35.9,38.0,12.15,31.7,13.29,19.20
4,16.00,314.42432,13.29,19.20,18.0,0.0241,5.18,0.070,1500.0,550.0,27.5,38.0,11.49,31.9,17.56,14.90
5,20.00,327.75680,17.56,14.90,20.0,0.0247,5.18,0.070,1500.0,550.0,20.8,38.0,10.75,32.4,22.11,12.00
6,24.00,337.80000,22.11,12.00,22.0,0.0340,5.18,0.069,1500.0,550.0,17.0,38.0,10.03,32.9,25.40,6.80
7,28.00,516.60000,25.40,6.80,22.0,0.0350,5.17,0.069,1500.0,550.0,14.6,38.0,9.34,33.2,29.26,1.82
8,32.55,677.70000,29.26,1.82,33.0,0.0468,5.18,0.071,1500.0,550.0,84.5,37.9,8.55,33.7,0.00,0.00
9,0.00,41.10000,1.82,34.50,10.0,0.0112,5.30,0.086,1500.0,550.0,77.7,38.0,8.51,31.7,4.71,31.75


## 模型训练和绘图

In [ ]:

plt.style.use('seaborn')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
# 在代码开头添加以下字体设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'Microsoft YaHei']  # 尝试多个中文字体
# 读取数据
df = filled_0_merged

# 准备特征和目标
X = df.iloc[:, :-2].values
y = df.iloc[:, -2:].values
target_names = df.columns[-2:].tolist()

# 数据标准化
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

# 修改创建序列的函数，每9行为一组，不重叠
def create_sequences(X, y, seq_length=9):
    X_seq, y_seq = [], []
    for i in range(0, len(X), seq_length):
        if i + seq_length <= len(X):
            X_seq.append(X[i:i+seq_length])
            y_seq.append(y[i:i+seq_length])
    return np.array(X_seq), np.array(y_seq)

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # 返回所有时间步的预测
        predictions = self.fc(lstm_out)
        return predictions

# 设置参数
seq_length = 9  # 修改为9
input_size = X.shape[1]
hidden_size = 64
num_layers = 2
output_size = y.shape[1]
learning_rate = 0.001
num_epochs = 50
batch_size = 1  # 修改为1，因为每个序列就是一个batch

# 创建序列数据
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# 创建数据加载器
dataset = TimeSeriesDataset(X_seq, y_seq)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 初始化模型
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 记录训练损失
losses = []

# 训练模型
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in data_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss/len(data_loader)
    losses.append(avg_loss)
    if (epoch + 1) % 10 == 0:
        print(f'轮次 {epoch+1}/{num_epochs}, 损失: {avg_loss:.4f}')

# 预测
model.eval()
with torch.no_grad():
    X_tensor = torch.FloatTensor(X_seq)
    predictions = model(X_tensor)
    predictions = predictions.reshape(-1, output_size)
    predictions = y_scaler.inverse_transform(predictions.numpy())
    actual = y_seq.reshape(-1, output_size)
    actual = y_scaler.inverse_transform(actual)

# 可视化训练过程
plt.figure(figsize=(12, 5))
plt.plot(losses)
plt.title('训练损失曲线')
plt.xlabel('训练轮次')
plt.ylabel('损失')
plt.savefig('step_9_training_loss.png')
plt.close()

# 预测结果对比折线图
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('预测结果对比')

# 创建时间序列索引
time_steps = np.arange(len(actual))

for i in range(2):
    # 绘制真实值
    axes[i].plot(time_steps, actual[:, i], 'b-', label='实际值', linewidth=2)
    # 绘制预测值
    axes[i].plot(time_steps, predictions[:, i], 'r--', label='预测值', linewidth=2)
    
    # 添加标题和标签
    axes[i].set_title(f'{target_names[i]}预测结果')
    axes[i].set_xlabel('时间步')
    axes[i].set_ylabel('值')
    axes[i].legend()
    axes[i].grid(True)

    # 添加网格
    axes[i].grid(True, linestyle='--', alpha=0.7)
    
    # 设置刻度
    axes[i].tick_params(axis='both', which='major', labelsize=10)

plt.tight_layout()
plt.savefig('step_9_prediction_results.png', dpi=300, bbox_inches='tight')
plt.close()

# 计算评估指标
mse = np.mean((predictions - actual) ** 2, axis=0)
mae = np.mean(np.abs(predictions - actual), axis=0)
mape = np.mean(np.abs((predictions - actual) / actual), axis=0) * 100

print("\n评估指标:")
for i in range(2):
    print(f"\n{target_names[i]}:")
    print(f"MSE: {mse[i]:.4f}")
    print(f"MAE: {mae[i]:.4f}")
    print(f"MAPE: {mape[i]:.2f}%")

## 测试数据

In [44]:

# test_excel_file = 'bio_test.xlsx'
# all_sheets = pd.read_excel(test_excel_file, sheet_name=None)

# # 转置每个工作表
# transposed_sheets = {}
# for sheet_name, df in all_sheets.items():
#     transposed_sheets[sheet_name] = df.transpose()
# # 将所有转置后的工作表保存到新的Excel文件
# with pd.ExcelWriter('test_transposed.xlsx') as writer:
#     for sheet_name, df in transposed_sheets.items():
#         df.to_excel(writer, sheet_name=sheet_name)

# print("所有工作表转置完成！")


所有工作表转置完成！


In [33]:

# 归一化batchsize样本大小 
# read_all_sheet(excel_file='test_transposed.xlsx',target_rows=5,writefile='test_merge.xlsx')

所有工作表已处理完成！每个工作表现在都是9行。


In [51]:
read_all_sheet('test_transposed.xlsx',9,'test_step_9.xlsx')

所有工作表已处理完成！每个工作表现在都是9行。


In [52]:
process_sheets('test_step_9.xlsx')
# 合并所有的sheet


处理完成！结果已保存到: processed_test_step_9.xlsx


In [55]:

# 读取Excel文件中的所有工作表
merged_excel_file = 'test_step_9.xlsx'
all_sheets = pd.read_excel(merged_excel_file, sheet_name=None)
# 初始化一个空的DataFrame来存储合并后的数据
merged_df = pd.DataFrame()

# 遍历所有工作表并合并
for sheet_name, df in all_sheets.items():
    # 假设发酵周期列名为'发酵周期'或'Time'，请根据实际情况调整
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.concat([merged_df, df], ignore_index=True)

# 保存合并后的数据到新的Excel文件
output_file = 'merged_test_step_9.xlsx'
merged_df.to_excel(output_file, index=False)

In [57]:
# 读取测试数据
test_file = 'merged_test_step_9.xlsx'
all_test_sheets = pd.read_excel(test_file, sheet_name=None)

# 存储所有预测结果
all_predictions = {}
all_actual = {}

# 对每个工作表进行预测
for sheet_name, test_df in all_test_sheets.items():
    # 准备测试数据
    X_test = test_df.iloc[:-2].values  # 除最后两行外的所有行作为特征
    y_test = test_df.iloc[-2:].values  # 最后两行作为实际值
    
    # 标准化测试数据（使用训练集的scaler）
    X_test_scaled = X_scaler.transform(X_test)
    
    # 重塑数据为LSTM所需的格式
    X_test_seq = X_test_scaled.T.reshape(1, 9, -1)  # 改变形状为(1, 9, features)
    
    # 使用模型进行预测
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test_seq)
        predictions = model(X_test_tensor)
        predictions = predictions.numpy().reshape(9, 2)  # 重塑为(9, 2)
        predictions = y_scaler.inverse_transform(predictions)  # 反标准化
    
    # 存储预测结果和实际值
    all_predictions[sheet_name] = predictions
    all_actual[sheet_name] = y_test.T

# 可视化每个工作表的预测结果
for sheet_name in all_test_sheets.keys():
    predictions = all_predictions[sheet_name]
    actual = all_actual[sheet_name]
    
    plt.figure(figsize=(15, 6))
    time_steps = range(len(predictions))
    
    # 绘制两个目标变量的预测结果
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(time_steps, actual[:, i], 'b-', label='实际值', linewidth=2)
        plt.plot(time_steps, predictions[:, i], 'r--', label='预测值', linewidth=2)
        
        plt.title(f'{sheet_name} - {target_names[i]}预测结果')
        plt.xlabel('发酵时间(h)')
        plt.ylabel('值')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xticks(time_steps)
    
    plt.tight_layout()
    plt.savefig(f'data/prediction_results_{sheet_name}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 计算并打印每个工作表的评估指标
    mse = np.mean((predictions - actual) ** 2, axis=0)
    mae = np.mean(np.abs(predictions - actual), axis=0)
    mape = np.mean(np.abs((predictions - actual) / actual), axis=0) * 100

    print(f"\n{sheet_name} 评估指标:")
    for i in range(2):
        print(f"\n{target_names[i]}:")
        print(f"MSE: {mse[i]:.4f}")
        print(f"MAE: {mae[i]:.4f}")
        print(f"MAPE: {mape[i]:.2f}%")

# 保存所有预测结果到Excel
with pd.ExcelWriter('data/test_prediction_results.xlsx') as writer:
    for sheet_name in all_test_sheets.keys():
        predictions = all_predictions[sheet_name]
        actual = all_actual[sheet_name]
        
        results_df = pd.DataFrame({
            f'{target_names[0]}_实际值': actual[:, 0],
            f'{target_names[0]}_预测值': predictions[:, 0],
            f'{target_names[1]}_实际值': actual[:, 1],
            f'{target_names[1]}_预测值': predictions[:, 1]
        }, index=[f'{i}h' for i in range(len(predictions))])
        
        results_df.to_excel(writer, sheet_name=sheet_name)

print("\n所有预测结果已保存到Excel文件中。")

ValueError: cannot reshape array of size 350 into shape (1,9,newaxis)